In [1]:
import os, sys
import glob
import math
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from argparse import ArgumentParser

#Covarage matrix as csv file without qoutes, seperated by commas
folder = "/home/pato/Skrivebord/HPV_subtyping/Results/"
topRunName = "Opsaet9_chip1_HPV16panel_1320_25112022"
indFile = "AnnotationIndividualFiles_"+topRunName+".txt"
sumFile = "AnnotationSummary_"+topRunName+".txt"

indFile = folder + topRunName + "/" + indFile
sumFile = folder + topRunName + "/" + sumFile

sublineagesFile = "/home/pato/Skrivebord/HPV_subtyping/References/ReferenceToSublineage.csv"

In [2]:
sumFile

'/home/pato/Skrivebord/HPV_subtyping/Results/Opsaet9_chip1_HPV16panel_1320_25112022/AnnotationSummary_Opsaet9_chip1_HPV16panel_1320_25112022.txt'

In [3]:
sumdf = pd.read_csv(sumFile, sep = "\t")
inddf = pd.read_csv(indFile, sep = "\t")
sublineages = pd.read_csv(sublineagesFile, sep = ",", header=None)

In [4]:
sublineages

,0,1,2,3,4
0,Alpha-10,HPV11,A,A1,M14119
1,Alpha-10,HPV11,A,A2,FN907962
2,Alpha-10,HPV11,A,A3,pending
3,Alpha-10,HPV11,A,A4,pending
4,Alpha-10,HPV11,B,B,pending
...,...,...,...,...,...
153,Alpha-5,HPV82,C,C3,KF436801
154,Alpha-5,HPV82,C,C4,KF436802
155,Alpha-5,HPV82,C,C5,KF436803
156,Alpha-7,HPV85,A,A1,AF131950


In [5]:
genes = list(sumdf['GENEID'].unique())
cols = ['Patient','Type','Reference matched','Sublineage']
cols.extend(genes)

In [6]:
for item in cols.copy():
    if "DoesNotExist" in item:
        cols.remove(item)

In [7]:
cols

['Patient',
 'Type',
 'Reference matched',
 'Sublineage',
 'gene-E1-71',
 'gene-E2-73',
 'E4_splice',
 'gene-E5-19',
 'gene-L2-74',
 'gene-L1-75',
 'E6',
 'E7',
 'E1',
 'E2',
 'E5_ALPHA',
 'L2',
 'L1']

In [8]:
# Changing genenames of ex gene-E4-172 to E4
newcols = []
for gene in cols:
    if "gene-" in gene:
        newgene = gene.split("-")[1]
        if newgene not in newcols:
            newcols.append(newgene)
    else:
        if gene not in newcols:
            newcols.append(gene)
cols = newcols

In [9]:
sumJson = []
for patient in inddf.columns:
    # Making json with col names
    ptjson = {}
    for col in cols:
        ptjson.update({
            col : ""
        })

In [10]:
spl = patient.split(":")

In [11]:
ref = spl[1]

In [12]:
ptjson

{'Patient': '',
 'Type': '',
 'Reference matched': '',
 'Sublineage': '',
 'E1': '',
 'E2': '',
 'E4_splice': '',
 'E5': '',
 'L2': '',
 'L1': '',
 'E6': '',
 'E7': '',
 'E5_ALPHA': ''}

In [21]:
sumJson = []
for patient in inddf.columns:
    # Making json with col names
    ptjson = {}
    for col in cols:
        ptjson.update({
            col : ""
        })
    
    # Getting patient name, reference and sublineage
    spl = patient.split(":")
    ref = spl[1]
    sublineage = ""
    type = ""
    # Getting sublineage class and type from the reference id
    
    subLineRow = 0
    for i in sublineages[4]:
        if i in ref:
            sublineage = sublineages[3][subLineRow]
            type = sublineages[1][subLineRow]
        subLineRow +=1

    ptjson.update({
        'Patient' : spl[0],
        'Type' : type,
        'Reference matched' : spl[1],
        'Sublineage' : sublineage
    })

    # For each row, get nuc and aa changes and match to gene key
    for row in inddf[patient]:
        # This check fails if value is NA
        if row == row:
            splt = row.split(" ")

            # If the cell contains a message such as "NoE4File", the column should be skipped. 
            if len(splt) == 2:
                continue

            if len(splt) == 4:

                nuc = splt[0]
                aa = splt[1]
                gene = splt[2]

                # Changing genenames of ex gene-E4-172 to E4
                if "gene-" in gene:
                    gene = gene.split("-")[1]
                
                if ptjson[gene] != "":
                    ptjson[gene].extend([[nuc,aa]])
                else:
                    ptjson.update({
                        gene : [[nuc,aa]]
                    })
            # Else if len is 5, there is a multi codon case. Because e.g c(36, 38) exists
            elif len(splt) == 5:
                nuc = splt[0]
                aa = str(splt[1:2]).replace(" ","")
                gene = splt[3]

                # Changing genenames of ex gene-E4-172 to E4
                if "gene-" in gene:
                    gene = gene.split("-")[1]
                if "_ALPHA" in gene:
                    gene = gene.split("_")[0]
                
                if ptjson[gene] != "":
                    ptjson[gene].extend([[nuc,aa]])
                else:
                    ptjson.update({
                        gene : [[nuc,aa]]
                    })

    #jsonlist = [ptjson]
    sumJson.append(ptjson)
sumJson

[{'Patient': '56.IonXpress_003',
  'Type': 'HPV16',
  'Reference matched': 'HPV16_K02718_1_revised',
  'Sublineage': 'A1',
  'E1': '',
  'E2': [['c.3205T>A', 'p.G150G'], ['c.3575T>G', 'p.S274A']],
  'E4_splice': [['c.3575T>G', 'p.A78A']],
  'E5': '',
  'L2': [['c.4419A>C', 'p.G61G'],
   ['c.4938G>A', 'p.Q234Q'],
   ['c.5226A>C', 'p.L330F']],
  'L1': '',
  'E6': '',
  'E7': '',
  'E5_ALPHA': ''},
 {'Patient': '136.IonXpress_014',
  'Type': 'HPV16',
  'Reference matched': 'HPV16_K02718_1_revised',
  'Sublineage': 'A1',
  'E1': [['c.1238G>A', 'p.R125K'],
   ['c.1252G>A', 'p.E130K'],
   ['c.1522T>A', 'p.S220T'],
   ['c.2457C>T', 'p.Y531Y']],
  'E2': [['c.3118T>C', 'p.F121F'],
   ['c.3384T>C', 'p.I210T'],
   ['c.3410C>T', 'p.P219S'],
   ['c.3684C>A', 'p.T310K']],
  'E4_splice': [['c.3384T>C', 'p.L15L'], ['c.3410C>T', 'p.T23T']],
  'E5': '',
  'L2': [['c.4461G>A', 'p.L75L'],
   ['c.4938G>A', 'p.Q234Q'],
   ['c.5041T>C', 'p.S269P'],
   ['c.5226A>C', 'p.L330F'],
   ['c.5281T>C', 'p.Y349H']],
 

In [17]:
sumJson[14]

{'Patient': '151.IonXpress_019',
 'Type': 'HPV16',
 'Reference matched': 'AF536179_1',
 'Sublineage': 'A2',
 'E1': ['c.1229CA>C', 'p.122', 'c.1295C>T', 'p.T144M', 'c.1884G>A', 'p.M340I'],
 'E2': ['c.2938G>A', 'p.T61T', 'c.3384C>T', 'p.T210I', 'c.3410T>C', 'p.S219P'],
 'E4_splice': ['c.3384C>T', 'p.T23I', 'c.3410T>C', 'p.S32P'],
 'E5': ['c.3979C>A', 'p.L44I'],
 'L2': ['c.4961G>A',
  'p.V243I',
  'c.5039C>T',
  'p.P269S',
  'c.5292T>C',
  'p.L353S'],
 'L1': ['c.5741T>C',
  'p.Y61Y',
  'c.5969G>GCCATTAGGTGTGCCACA',
  'p.137',
  'c.6524C>T',
  'p.S322S',
  'c.6992A>G',
  'p.K478K'],
 'E6': '',
 'E7': '',
 'E5_ALPHA': ''}

In [22]:
formDf = pd.DataFrame.from_dict(sumJson)
formDf

,Patient,Type,Reference matched,Sublineage,E1,E2,E4_splice,E5,L2,L1,E6,E7,E5_ALPHA
0,56.IonXpress_003,HPV16,HPV16_K02718_1_revised,A1,,"[[c.3205T>A, p.G150G], [c.3575T>G, p.S274A]]","[[c.3575T>G, p.A78A]]",,"[[c.4419A>C, p.G61G], [c.4938G>A, p.Q234Q], [c...",,,,
1,136.IonXpress_014,HPV16,HPV16_K02718_1_revised,A1,"[[c.1238G>A, p.R125K], [c.1252G>A, p.E130K], [...","[[c.3118T>C, p.F121F], [c.3384T>C, p.I210T], [...","[[c.3384T>C, p.L15L], [c.3410C>T, p.T23T]]",,"[[c.4461G>A, p.L75L], [c.4938G>A, p.Q234Q], [c...","[[c.6295A>G, p.E219E], [c.6391A>G, p.R251R], [...",,"[[c.822A>G, p.L87L]]","[[c.3979A>C, p.I44L], [c.4042A>G, p.I65V]]"
2,152.IonXpress_020,HPV16,HPV16_K02718_1_revised,A1,"[[c.1053A>C, p.E63D], [c.2376T>C, p.S504S]]","[[c.3058G>A, p.V101V], [c.3410C>T, p.P219S]]","[[c.3410C>T, p.T23T]]",,"[[c.4938G>A, p.Q234Q], [c.5226A>T, p.L330F], [...","[[c.6434A>G, p.T266A], [c.7060G>A, p.L474L]]","[[c.350T>G, p.L83V]]",,"[[c.3979A>C, p.I44L], [c.4042A>G, p.I65V]]"
3,48_rec.IonXpress_002,HPV16,HPV16_K02718_1_revised,A1,"[[c.1522T>A, p.S220T], [c.2457C>T, p.Y531Y]]","[[c.3118T>C, p.F121F], [c.3176G>A, p.E141K], [...","[[c.3384T>C, p.L15L], [c.3410C>T, p.T23T]]",,"[[c.4938G>A, p.Q234Q], [c.5139T>A, p.I301I], [...","[[c.6434A>G, p.T266A]]","[[c.350T>G, p.L83V]]",,"[[c.3979A>C, p.I44L]]"
4,135.IonXpress_013,HPV16,HPV16_K02718_1_revised,A1,"[[c.1000GAAGATTT>G, ['p.c(46,']]]",,,,"[[c.4290T>G, p.L18L], [c.4938G>A, p.Q234Q], [c...","[[c.5743T>C, p.Y35Y], [c.6434A>G, p.T266A], [c...",,,
5,79.IonXpress_007,HPV16,HPV16_K02718_1_revised,A1,"[[c.1053A>C, p.E63D]]","[[c.3410C>T, p.P219S]]","[[c.3410C>T, p.T23T]]",,"[[c.4938G>A, p.Q234Q], [c.5226A>T, p.L330F]]","[[c.6434A>G, p.T266A]]","[[c.109T>C, p.F2F], [c.350T>G, p.L83V]]",,"[[c.3979A>C, p.I44L], [c.4042A>G, p.I65V]]"
6,48.IonXpress_001,HPV16,HPV16_K02718_1_revised,A1,"[[c.1229CA>C, p.122], [c.1522T>A, p.S220T]]","[[c.3176G>A, p.E141K]]",,,"[[c.4938G>A, p.Q234Q], [c.5139T>A, p.I301I], [...","[[c.6434A>G, p.T266A]]","[[c.350T>G, p.L83V]]",,"[[c.3979A>C, p.I44L]]"
7,66.IonXpress_005,HPV16,HPV16_K02718_1_revised,A1,"[[c.1522T>A, p.S220T], [c.2457C>T, p.Y531Y]]","[[c.3118T>C, p.F121F], [c.3384T>C, p.I210T], [...","[[c.3384T>C, p.L15L], [c.3410C>T, p.T23T]]",,"[[c.4461G>A, p.L75L], [c.4938G>A, p.Q234Q], [c...","[[c.6391A>G, p.R251R], [c.6434A>G, p.T266A]]",,"[[c.822A>G, p.L87L]]","[[c.3979A>C, p.I44L], [c.4042A>G, p.I65V]]"
8,101_rec.IonXpress_009,HPV16,HPV16_K02718_1_revised,A1,"[[c.940GA>G, p.26], [c.1871C>T, p.T336M]]",,,,"[[c.4738G>A, p.V168I], [c.4938G>A, p.Q234Q], [...","[[c.5831G>T, p.V65L]]",,,
9,108.IonXpress_010,HPV16,HPV16_K02718_1_revised,A1,,,,,"[[c.4938G>A, p.Q234Q], [c.4963G>A, p.V243I], [...","[[c.5962T>C, p.G108G], [c.6434A>G, p.T266A]]","[[c.350T>G, p.L83V]]",,"[[c.3979A>C, p.I44L], [c.4042A>G, p.I65V]]"


In [49]:
# Convert to dataframe and save to file
filename = folder+"/"+topRunName+"/AnnotationFormatted_"+topRunName+".txt"
formDf.to_csv(filename, index = False)